Implementacja modelu w PyTorch

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

class MultiInputModel(nn.Module):
    def __init__(self, num_classes):
        super(MultiInputModel, self).__init__()
        
        # Sieć dla obrazów RGB (widok T i B) - EfficientNet-b0
        self.rgb_model = models.efficientnet_b0(pretrained=True)
        self.rgb_model.classifier = nn.Identity()  # Usuń ostatnią warstwę (1280 cech)

        # Sieć dla obrazu binarnego (widok S)
        self.binary_model = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),  # 1-kanałowe wejście
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(32 * (224 // 4) * (224 // 4), 256),  # Dopasowanie rozmiarów
            nn.ReLU()
        )

        # Warstwa łącząca
        self.fc = nn.Sequential(
            nn.Linear(1280 + 1280 + 256, 512),  # 1280 (T) + 1280 (B) + 256 (S)
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, t_image, b_image, s_image):
        # Ekstrakcja cech dla widoków RGB
        t_features = self.rgb_model(t_image)  # Widok T
        b_features = self.rgb_model(b_image)  # Widok B

        # Ekstrakcja cech dla obrazu binarnego
        s_features = self.binary_model(s_image)

        # Połączenie cech
        combined_features = torch.cat([t_features, b_features, s_features], dim=1)

        # Klasyfikacja
        output = self.fc(combined_features)
        return output


Krok 1: Przygotowanie danych. Przygotuj dane dla modelu multi-input w formie DataLoader, gdzie każda próbka zawiera trzy obrazy (*_T.png, *_B.png, *_S.png) i ich klasę.

In [2]:
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd

class MultiInputDataset(Dataset):
    def __init__(self, csv_file, transform_rgb=None, transform_binary=None):
        self.data = pd.read_csv(csv_file)

        # Tworzenie mapowania nazw klas na liczby całkowite
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(self.data['class'].unique())}

        self.transform_rgb = transform_rgb
        self.transform_binary = transform_binary

    def __len__(self):
        return len(self.data) // 3  # Każde ziarno ma 3 obrazy

    def __getitem__(self, idx):
        # Pobierz trzy obrazy
        base_idx = idx * 3
        t_path = self.data.iloc[base_idx]['path']
        b_path = self.data.iloc[base_idx + 1]['path']
        s_path = self.data.iloc[base_idx + 2]['path']

        t_image = Image.open(t_path).convert("RGB")
        b_image = Image.open(b_path).convert("RGB")
        s_image = Image.open(s_path).convert("L")  # Obraz binarny

        # Transformacje
        if self.transform_rgb:
            t_image = self.transform_rgb(t_image)
            b_image = self.transform_rgb(b_image)
        if self.transform_binary:
            s_image = self.transform_binary(s_image)

        # Pobierz nazwę klasy i przekształć na indeks numeryczny
        class_name = self.data.iloc[base_idx]['class']
        label = self.class_to_idx[class_name]  # Mapowanie nazwy klasy na numer
        label = torch.tensor(label, dtype=torch.long)  # Konwersja na tensor PyTorch

        return t_image, b_image, s_image, label




Krok 2: Transformacje danych
Transformacje dla obrazów RGB i binarnych:

In [3]:
from torchvision import transforms

# Transformacje dla obrazów RGB
transform_rgb = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Transformacje dla obrazów binarnych
transform_binary = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


Krok 3: Trenowanie modelu

In [ ]:
from torch.utils.data import DataLoader
import torch.optim as optim
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Załaduj dane
train_dataset = MultiInputDataset("CSV/dataset/train.csv", transform_rgb=transform_rgb, transform_binary=transform_binary)
val_dataset = MultiInputDataset("CSV/dataset/val.csv", transform_rgb=transform_rgb, transform_binary=transform_binary)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Inicjalizacja modelu
model = MultiInputModel(num_classes=11)  # Liczba klas
model = model.to("cuda")  # Jeśli używasz GPU

# Optymalizator i funkcja straty
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Pętla treningowa
for epoch in range(10):
    model.train()
    total_loss = 0
    for t_image, b_image, s_image, labels in train_loader:
        t_image, b_image, s_image, labels = (
            t_image.to("cuda"),
            b_image.to("cuda"),
            s_image.to("cuda"),
            labels.to("cuda")
        )

        # Oblicz predykcje i stratę
        outputs = model(t_image, b_image, s_image)

        #print(f"Outputs shape: {outputs.shape}")  # Dodaj tę linię
        #print(f"Labels shape: {labels.shape}")    # Dodaj tę linię
        #print(f"Labels min: {labels.min()}, Labels max: {labels.max()}")  # Dodaj tę linię
        #print(f"Labels dtype: {labels.dtype}")  # Dodaj tę linię

        loss = criterion(outputs, labels)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()


    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader):.4f}")


/home/mk/miniconda3/envs/dnn_gpu/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mk/miniconda3/envs/dnn_gpu/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Outputs shape: torch.Size([32, 11])
Labels shape: torch.Size([32])
Labels min: 0, Labels max: 10
Labels dtype: torch.int64
Outputs shape: torch.Size([32, 11])
Labels shape: torch.Size([32])
Labels min: 0, Labels max: 10
Labels dtype: torch.int64
Outputs shape: torch.Size([32, 11])
Labels shape: torch.Size([32])
Labels min: 0, Labels max: 10
Labels dtype: torch.int64
Outputs shape: torch.Size([32, 11])
Labels shape: torch.Size([32])
Labels min: 0, Labels max: 10
Labels dtype: torch.int64
Outputs shape: torch.Size([32, 11])
Labels shape: torch.Size([32])
Labels min: 0, Labels max: 10
Labels dtype: torch.int64
Outputs shape: torch.Size([32, 11])
Labels shape: torch.Size([32])
Labels min: 0, Labels max: 10
Labels dtype: torch.int64
Outputs shape: torch.Size([32, 11])
Labels shape: torch.Size([32])
Labels min: 0, Labels max: 10
Labels dtype: torch.int64
Outputs shape: torch.Size([32, 11])
Labels shape: torch.Size([32])
Labels min: 0, Labels max: 10
Labels dtype: torch.int64
Outputs shape: t